In [1]:
!pip install -q html2text lxml_html_clean

In [1]:
from fastcore.all import Path
from fastcore.parallel import parallel
import pandas as pd
from typing import List, Generator, Optional
from lxml.html import HtmlElement, fromstring, tostring
from lxml_html_clean import Cleaner
import traceback
from html2text import HTML2Text
import requests
import lxml
from lxml.html import HtmlElement, tostring, fromstring
from fastcore.parallel import parallel
from urllib.parse import  unquote, urlparse
import regex as re
from lxml import html
import re
from datetime import datetime

cleaner = Cleaner(javascript=True, style=True)

def html_repr(self):
    return tostring(self, encoding='unicode', pretty_print=True)

HtmlElement.__repr__ = html_repr
HtmlElement.__str__ = html_repr
path = Path('.').cwd()

### Logger

In [2]:
import logging
import traceback

def setup_logging(log_file: str = "extract_article_app.log"):
    # Create a custom logger
    logger = logging.getLogger("LinkExtractor")
    logger.setLevel(logging.INFO)  # Capture only INFO and above (INFO, WARNING, ERROR, CRITICAL)

    # Prevent logger duplication
    if not logger.hasHandlers():
        # Create file handler to log to a file
        file_handler = logging.FileHandler(log_file)
        file_handler.setLevel(logging.INFO)  # File handler also captures only INFO and above

        # Create log format with function name included
        log_format = logging.Formatter('%(asctime)s - %(levelname)s - [%(funcName)s] - %(message)s')
        file_handler.setFormatter(log_format)

        # Add handler to the logger
        logger.addHandler(file_handler)

    return logger

# Step 2: Set up logging to a file only, suppressing output in the notebook
logger = setup_logging(log_file="notebook_logs.log")

# Step 3: Example usage
logger.info("This is an INFO log message.")
logger.debug("This DEBUG message won't appear in INFO mode.")
logger.warning("This is a WARNING log message.")

### Extract csv files

In [3]:
df = pd.read_csv('links.csv')
df.head()

,article_link
0,https://www.dharitri.com/in-this-india-466/
1,https://www.dharitri.com/survive-is-difficulty/
2,https://www.dharitri.com/in-this-india-697/
3,https://www.dharitri.com/explainer-of-dual-cul...
4,https://www.dharitri.com/ପ୍ରତିବାଦ-ପ୍ରତିରୋଧ-ନୁହେଁ/


### Webscrapper

In [4]:

def get_title_date(txt):
    """
    get the title and date of the article
    the input is like : '\n                    ଏଇ ଭାରତରେ                    \n                    \t\t\t\t\t\t   \n                        7:03 amJanuary 9, 2022January 9, 2022 / \n\t\t\t\t\t\t21\t\t\t\t\t\t\n\n                    \n\n                '
    """
    lines = re.findall(r'(?<=\n)\s+(.+?)\s+(?=\n)', txt)
    if len(lines) >= 2: 
        datetime_match = re.search(r'(\d{1,2}:\d{2}\s*[ap]m.*?\d{4})', lines[1])
        datetime_str = datetime_match.group() if datetime_match else None
        
        # Convert datetime string to datetime object if found
        datetime_obj = None
        if datetime_str:
                datetime_obj = datetime.strptime(datetime_str, '%I:%M %p%B %d, %Y')
        return lines[0], datetime_obj
    if len(lines) == 1:
        return lines[0], None
    return None, None
s = '\n                    ଏଇ ଭାରତରେ                    \n                    \t\t\t\t\t\t   \n                        7:03 amJanuary 9, 2022January 9, 2022 / \n\t\t\t\t\t\t21\t\t\t\t\t\t\n\n                    \n\n                '
get_title_date(s)

('ଏଇ ଭାରତରେ', datetime.datetime(2022, 1, 9, 7, 3))

In [ ]:
pttr = {
    'url_patrn': r"(ht?t?p?s?|ftp):?\/\/\S+",
    'mobile_ptrn': r"ମୋ\W*[୦-୯]{10}\W*",
    'email_ptrn': r"(?i)((email)?[:\s\-]*\S+@\S+\.\S+)",
    'eng_stmt': r"^\s*(?:\d+\.\s*|\-|\•|\*)?\s*[A-Za-z0-9\s,;:'\"()!?-]*[.?!]\s*$",
    
}

class WebScraper:
    def __init__(self, url: str, base_xpath:str, title_xpath:str):
        """Initialize scraper with URL and extract body."""
        self.url = url
        self.title, self.date, self.body = None, None, None
        self._get_body(base_xpath, title_xpath)

    def _get_body(self, base_xpath:str, title_xpath:str) -> Optional[HtmlElement]:
        """Fetch and clean HTML body from URL."""
        try:
            logger.debug(f"Fetching URL: {self.url}")
            #response = httpx.get(self.url)
            response = requests.get(self.url, timeout=50)
            html_tree = fromstring(response.text)

            self.title, self.date = get_title_date(html_tree.xpath(title_xpath)[0].text_content())

            body = html_tree.xpath('//body')[0]
            assert isinstance(body, HtmlElement)
            
            logger.debug("Cleaning HTML body")
            
            cleaned_body = cleaner.clean_html(body)
            
            paras = fromstring(''.join(
                tostring(c, encoding='unicode') for c in cleaned_body
            )).xpath(base_xpath)
            
            div = html.Element('div'); [div.append(p) for p in paras if bool(p.text_content().strip())]
            self.body = div
            
            logger.info(f"Successfully processed URL: {self.url}")
            return 
        
        except Exception as e:
            logger.error(f"Error processing URL {self.url}: {str(e)}")
            logger.error(traceback.format_exc())
            return None


      
    def to_markdown(self, body_width: int = 5000000) -> Optional[str]:
        """Convert the processed HTML body to markdown."""
        if self.body is None:
            logger.error(f"Body is None, cannot convert to markdown for : {self.url}")
            return ''

        try:
            logger.debug("Converting HTML to markdown")
            h2t = HTML2Text(bodywidth=body_width)
            h2t.ignore_links = True
            h2t.mark_code = True
            h2t.ignore_images = True
            
            
            html_string = tostring(self.body, encoding='unicode')
            markdown_text = h2t.handle(html_string).replace('\u200c', '')
            
            for p in pttr.values():
               markdown_text = re.sub(p, "", markdown_text).strip()
                
            logger.info(f"Successfully converted HTML to markdown of length {len(markdown_text)}")
            return markdown_text
        except Exception as e:
            logger.error(f"Error converting to markdown: {str(e)} : {self.url}")
            logger.error(traceback.format_exc())
            return None
            
    def get_processed_body(self) -> Optional[HtmlElement]:
        """Return the processed body."""
        return self.body

In [32]:
url = 'https://www.dharitri.com/ironically/'
scraper = WebScraper(url, base_xpath='//div[@class="p-t-20"]/child::p', title_xpath='//h1[@class="my_menu "]')
print(scraper.to_markdown())

ଉପେନ୍ଦ୍ର ବିଶ୍ୱାଳ

୫ ବର୍ଷ ପୂର୍ବର ଏକ ଘଟଣା। ବ୍ରିଟିଶ୍ ଶାସକଙ୍କ ଦ୍ୱାରା ଭାରତକୁ କ୍ଷମତା ହସ୍ତାନ୍ତରର ବିରୋଧ କରୁଥିଲେ ଉଇନ୍ଷ୍ଟନ ଚର୍ଚ୍ଚିଲ। ବିଦ୍ରୂପଭରା ସେ ଯେଉଁ ମନ୍ତବ୍ୟ ରଖିଥିଲେ ତାହାର ସଂକ୍ଷିପ୍ତ ସାରାଂଶ ହେଲା- ଇଣ୍ଡିଆକୁ ସ୍ବାଧୀନତା ଦେଲେ ଅବସ୍ଥା ଆହୁରି ବିଗିଡ଼ିଯିବ। ଦିନେ ଏମିତି ସ୍ଥିତି ଆସିବ, ସେ ଦେଶରେ ପାଣି, ପବନ ବି ବିକ୍ରି ହେବ। ସେ ଦିନର ସେ ମର୍ମଭେଦୀ କଟାକ୍ଷ ଏକପ୍ରକାର ସତ୍ୟ ସାବ୍ୟସ୍ତ ହୋଇଛି ଆଜି। ପାଣି ତ କେବେଠୁ ବିକ୍ରି ଚାଲିଛି ଆଉ ପବନ (ଅମ୍ଳଜାନ) ବି ଆଜି ବିକ୍ରି ହେଉଛି କଳା ବଜାରରେ। ବାସ୍ତବରେ ଚର୍ଚ୍ଚିଲଙ୍କ ଏମିତି ଏକ ଶକ୍ତ ଚଟକଣା ଯେମିତି ଅପମାନ ଓ ଅବସାଦଭରା, ସେମିତି ହୃଦୟବିଦାରକ ଓ ବିଡ଼ମ୍ବନା ମଧ୍ୟ। ସ୍ବାଧୀନତାର ସେ ସମୟ ଥିଲା ସମ୍ଭାବନାମୟ ବେଳାଟିଏ ଆଗକୁ ଆଗେଇ ଯିବାର। ସମୟ ଓ ସୁଯୋଗ ବି ହାତଠାରି ଆମକୁ ଡାକୁଥିଲା ଭାରତ ଓ ଭାରତୀୟତାକୁ ଉଦ୍ଭାସିତ କରିବାକୁ। କିନ୍ତୁ ଦୁର୍ଭାଗ୍ୟ, ଦେଶକୁ ମିଳିଲାନି ସଠିକ୍ ନେତୃତ୍ୱ ଓ ଦିଗ୍ଦର୍ଶନ। ଫଳତଃ ଦିଶାହୀନ ହେଲା ଦେଶ। ତା’ ନ ହୋଇଥିଲେ ସ୍ବାଧୀନତାର ୫୦ ବର୍ଷ ଭିତରେ ଦେଶରେ ୨ଟିରୁ ଅଧିକ ଏମ୍ସ କଲେଜ କ’ଣ ପ୍ରତିଷ୍ଠା ହୋଇ ପାରିନଥାନ୍ତା? ଦେଶର ଭବିଷ୍ୟତ ଯେମିତି ନିର୍ଭରକରେ ଲୋକ ଚରିତ୍ର ଉପରେ, ସେମିତି ବି ନିର୍ଭର କରେ ନେତୃତ୍ୱର ପରାକାଷ୍ଠା ଓ ପାରଦର୍ଶିତା ଉପରେ। ବରଂ ଦକ୍ଷ ଓ ଦୃଢ଼ମନା ନେତୃତ୍ୱ ସକ୍ଷମ ହୁଏ ଜନ ମାନସିକତା

In [33]:
url = 'https://www.dharitri.com/ମମତାର-ଡାକ/'
scraper = WebScraper(url, base_xpath='//div[@class="p-t-20"]/child::p', title_xpath='//h1[@class="my_menu "]')
print(scraper.to_markdown())

ଦ୍ୱିତୀ ଚନ୍ଦ୍ର ସାହୁ  
ପ୍ରବଳ ଖରାରେ ଗାଁକୁ ଯିବାକୁ ବାହାରିଲେ ମା’। ସ୍ନେହର ପରଶରେ, ମମତାର ମୋହରେ ବନ୍ଧା ସ୍କୁଲପଢୁଆ କିଶୋର ଛାଡ଼ିବାକୁ ବାହାରିଲା ଆପେ ଆପେ। ପୁଅ ଧରିଲା ସାଇକେଲ। ନିମିଷକରେ ପହଞ୍ଚିଗଲେ ବସ୍ଷ୍ଟପରେ। ଫେରିବାବେଳେ ରାସ୍ତାର ଏକ ମୋଡ଼ରେ ଦ୍ରୁତଗତିରେ ଆସୁଥିବା ଟ୍ରକ୍ ତଳେ ଚାପି ହୋଇଗଲା ପୁଅ ଆଖିପିଛୁଳାକେ। ଘଟଣା ସମ୍ପର୍କରେ ଜାଣିବା ପରେ ମା’ଙ୍କ ଆର୍ତ୍ତ ଚିତ୍କାରରେ ଫାଟିପଡ଼ିଲା ଘଟଣାସ୍ଥଳ। ମରଣ ସହ ସଂଘର୍ଷ କରୁଥିବା ପୁଅକୁ ବଞ୍ଚାଇବା ଲାଗି ଆକୁଳ ନିବେଦନ କଲା ସେ। ରାସ୍ତାରେ ଯାଉଥିବା ଅନ୍ୟ ଗାଡ଼ି ଓ ସମବେତ ଲୋକଙ୍କ ନିକଟରେ ନେହୁରା ହେଲା ନିଜ ପୁଅକୁ ଉଦ୍ଧାର କରିବା ଲାଗି। ଶେଷରେ ଉଦ୍ଧାର ହେଲା ପୁଅ। କିନ୍ତୁ ବିଳମ୍ବଜନିତ କାରଣରୁ ମା’କୁ ଚିରଦିନ ଛାଡି ଚାଲିଗଲା କିଶୋର ପୁଅ। ଏ ଯାଏ ପୁଅର ଜୀବନ ଲାଗି ସଂଗ୍ରାମ କରୁଥିବା ମା’ ଜୀବନର ନିଷ୍ଠୁର ସତ୍ୟକୁ ସାମ୍ନା କରିବା ପରେ ଚେତାଶୂନ୍ୟ ହୋଇଗଲା। ସେ ଜାଣିଗଲା ତା’ ଆଖି ସାମ୍ନାରେ ପୁଅ କୋଳ ଶୂନ୍ୟକରି ଭଗବାନଙ୍କ ପ୍ରିୟ ହୋଇଗଲା।  
ଆଉ ଏକ ଦୃଶ୍ୟ। ୯୯ ବର୍ଷ ବୟସ୍କା ମା’ ହୀରାବେନ୍ ମୋଦି। ପ୍ରଧାନମନ୍ତ୍ରୀ ନରେନ୍ଦ୍ର ମୋଦିଙ୍କ ମା’। ପୁତ୍ର ପ୍ରଚଣ୍ଡ ଜନାଦେଶ ପାଇ ଦ୍ୱିତୀୟଥର ନିର୍ବାଚିତ ହେବା ପରେ ମା’ଙ୍କ ଆଶୀର୍ବାଦ କାମନା ଲାଗି ଛୁଟିଗଲେ ଗାନ୍ଧୀନଗର। ଯାହାଙ୍କ ନିକଟରେ ସମସ୍ତେ ମଥାନତ ହେଉଛନ୍ତି, ସେ କିନ୍ତୁ ନତମସ୍ତକପୂର୍ବକ ମା’ଙ୍କ ପ

### crawl

In [8]:
link_lis = df['article_link'].tolist()
len(link_lis)

243240

In [9]:
def crawl(url):
    scrap = WebScraper(url, base_xpath='//div[@class="p-t-20"]/p', title_xpath='//h1[@class="my_menu "]')
    logger.info(f"{scrap.title=} {scrap.date=}")
    return scrap.title, scrap.date, scrap.to_markdown()
crawl(link_lis[0])

('ଏଇ ଭାରତରେ',
 datetime.datetime(2022, 1, 9, 7, 3),
 'ପଶ୍ଚିମବଙ୍ଗର ସୁନ୍ଦରବନରେ ସଂକ୍ରମିତଙ୍କ ଚିକିତ୍ସା ଲାଗି ଆବଶ୍ୟକ ଅକ୍ସିଜେନ ଯୋଗାଇବାରେ ଲାଗିଛନ୍ତି ସୌମିତ୍ର ମଣ୍ଡଳ। ବଡ଼କଥା ହେଉଛି, ସୁନ୍ଦରବନରେ ଥିବା ୯ଟି ଦ୍ୱୀପାଞ୍ଚଳରେ ରାସ୍ତା ଭଲ ନ ଥିତ୍ବାରୁ ସେଠାକାର ପୀଡ଼ିତଙ୍କ ନିକଟରେ ସେ ସାଇକେଲରେ ନେଇ ଅକ୍ସିଜେନ ପହଞ୍ଚାଉଛନ୍ତି। ସେଥିପାଇଁ ତାଙ୍କୁ ସୁନ୍ଦରବନର ଅକ୍ସିଜେନ ମଣିଷ କୁହାଯାଉଛି। ମଣ୍ଡଳଙ୍କ ବୟସ ୨୯ ଓ ସେ ଏକୁଟିଆ ସାଇକେଲରେ ଦୁର୍ଗମ ଅଞ୍ଚଳର ଗାଁ ଗାଁ ବୁଲି ଲୋକଙ୍କୁ ଓ୍ୟଷଧ ଓ ଅକ୍ସିଜେନ ଠିକ୍ ସମୟରେ ଦେଉଛନ୍ତି। ସେ ସାମାଜିକ କର୍ମୀ ଓ ଏନ୍ଜିଓଗୁଡ଼ିକଠାରୁ ଅକ୍ସିଜେନ ସିଲିଣ୍ଡର ସଂଗ୍ରହ କରୁଛନ୍ତି। ସ୍ଥାନୀୟ ପ୍ରଶାସନ ମଧ୍ୟ ତାଙ୍କୁ ଏ ଦିଗରେ ସହଯୋଗ କରୁଛି। ଶିକ୍ଷା ଶେଷ କରି ପ୍ରଥମେ ସେ ନିଜ ଗାଁ ଗୋସାବାର ସରକାରୀ ସ୍କୁଲରେ ଅସ୍ଥାୟୀ ଶିକ୍ଷକ ଭାବେ ଯୋଗ ଦେଇଥିଲେ। ସେଥିତ୍ରେ ସ୍ଥାୟୀ ଶିକ୍ଷକ ନିଯୁକ୍ତ ହେବା ପରେ ୨୦୧୯ରେ ସେ ଚାକିରି ହରାଇଥିଲେ। ପରେ ସେ ପିଲାମାନଙ୍କୁ ମାଗଣାରେ ଟ୍ୟୁଶନ କରିବା ସହ ଏହି କାର୍ଯ୍ୟ ଆରମ୍ଭ କରିଥିଲେ, ଯାହାକୁ ଏଯାବତ୍ ସେ ଚାଲୁ ରଖିଛନ୍ତି।')

In [18]:
crawl('https://www.dharitri.com/the-value-of-the-human-head/')

('ମଣିଷ ମୁଣ୍ଡର ମୂଲ୍ୟ',
 datetime.datetime(2020, 10, 18, 6, 50),
 '**ଚିତ୍ର ଚରିତ୍ର/ ଅଧ୍ୟାପକ ନିରଞ୍ଜନ ପାଢ଼ୀ**\n\nଜଧାନୀର ରାଜରାସ୍ତା ଉପରେ ହାତରେ ଖଣ୍ଡାଧରି ନିର୍ମମ ଭାବେ ଜଣେ ନିରୀହ ଲୋକକୁ ହାଣି ପକାଉଥିବା ସେଇ ଦୁର୍ଦ୍ଧର୍ଷ ଆତତାୟୀକୁ ଯେଉଁମାନେ ଟିଭି ପରଦାରେ ଅବା ସାମାଜିକ ଗଣମାଧ୍ୟମରେ ଦେଖିଛନ୍ତି ଏବଂ ପରବର୍ତ୍ତୀ ପର୍ଯ୍ୟାୟରେ ତାକୁ ସୁଦୂର ତୁମୁଡିବନ୍ଧରେ ଲୁଚିଥିବା ବେଳେ ସେଠାରୁ ଧରିଆଣି ପୋଲିସ ଜେରା କରିବା ଘଟଣା ଉପରେ ନଜର ରଖିଛନ୍ତି, ସେମାନେ ଜାଣିବେ ଜଣେ ମଣିଷ ମୁଣ୍ଡର ମୂଲ୍ୟ କେତେ! ପ୍ରଥମେ ଦଶ ହଜାର ଓ ମୁଣ୍ଡ କାଟି ସାରିଲା ପରେ ବାକି ଚାଳିଶ ହଜାର, ଏମିତି ସର୍ବମୋଟ ପଚାଶ ହଜାର ଟଙ୍କା ହେଉଛି ସେଇ ହତଭାଗ୍ୟ ମିଠା ଦୋକାନୀଟିର ମୁଣ୍ଡର ମୂଲ୍ୟ। ଅବଶ୍ୟ ଏ ପ୍ରକାର ଦର ନିର୍ଦ୍ଧାରଣ ସ୍ଥାନ, କାଳ ଓ ପାତ୍ର ବିଶେଷରେ ଭିନ୍ନ ଭିନ୍ନ ହୋଇଥିବ ବୋଲି ବିଶ୍ୱାସ କରାଯାଏ। ରାସ୍ତାକଡ଼ରେ ମିଠା ବିକୁଥିବା ମାମୁଲି ଲୋକଟିର ମୁଣ୍ଡ ଯଦି ପଚାଶ ହଜାର ହୁଏ ତେବେ ଶୀତତାପ ନିୟନ୍ତ୍ରିତ ସୁନା ଦୋକାନ ମାଲିକଙ୍କର ମୂଲ୍ୟ ନିଶ୍ଚୟ ସେହି ଅନୁସାରେ କେଇ ଲକ୍ଷ ହୋଇଥିବ !!  \nଏହି ଦୁର୍ଭାଗ୍ୟପୂର୍ଣ୍ଣ ଘଟଣାଟି ଶେଷ ହୋଇଗଲା ସିନା ଜଣେ ଗରିବ ରାସ୍ତାକଡ ଦୋକାନୀର ମୁଣ୍ଡ ଗଡିବାରେ, ମାତ୍ର ଆରମ୍ଭ ହୋଇଥିଲା ଦୁଇ ପ୍ରକାର ମୁଣ୍ଡର ମୂଲ୍ୟ ନିର୍ଦ୍ଧାରଣରେ। ହତ୍ୟାର ଶିକାର ହୋଇଥିବା କଥିତ ଦୋକାନୀଟି ପ୍ରଥମେ ଥିଲା 

In [10]:
import os; os.cpu_count()

8

In [11]:
def create_op(fn):
    final = [[url, lis[0], lis[1], lis[2]] for url, lis in zip(link_lis[prev:now], result)]
    df = pd.DataFrame(final, columns=['url', 'title', 'time', 'data'])
    print(df.head())
    df.to_csv(fn, index=False)

In [26]:
crawl(link_lis[0])

('ଏଇ ଭାରତରେ',
 datetime.datetime(2022, 1, 9, 7, 3),
 'ପଶ୍ଚିମବଙ୍ଗର ସୁନ୍ଦରବନରେ ସଂକ୍ରମିତଙ୍କ ଚିକିତ୍ସା ଲାଗି ଆବଶ୍ୟକ ଅକ୍ସିଜେନ ଯୋଗାଇବାରେ ଲାଗିଛନ୍ତି ସୌମିତ୍ର ମଣ୍ଡଳ। ବଡ଼କଥା ହେଉଛି, ସୁନ୍ଦରବନରେ ଥିବା ୯ଟି ଦ୍ୱୀପାଞ୍ଚଳରେ ରାସ୍ତା ଭଲ ନ ଥିତ୍ବାରୁ ସେଠାକାର ପୀଡ଼ିତଙ୍କ ନିକଟରେ ସେ ସାଇକେଲରେ ନେଇ ଅକ୍ସିଜେନ ପହଞ୍ଚାଉଛନ୍ତି। ସେଥିପାଇଁ ତାଙ୍କୁ ସୁନ୍ଦରବନର ଅକ୍ସିଜେନ ମଣିଷ କୁହାଯାଉଛି। ମଣ୍ଡଳଙ୍କ ବୟସ ୨୯ ଓ ସେ ଏକୁଟିଆ ସାଇକେଲରେ ଦୁର୍ଗମ ଅଞ୍ଚଳର ଗାଁ ଗାଁ ବୁଲି ଲୋକଙ୍କୁ ଓ୍ୟଷଧ ଓ ଅକ୍ସିଜେନ ଠିକ୍ ସମୟରେ ଦେଉଛନ୍ତି। ସେ ସାମାଜିକ କର୍ମୀ ଓ ଏନ୍ଜିଓଗୁଡ଼ିକଠାରୁ ଅକ୍ସିଜେନ ସିଲିଣ୍ଡର ସଂଗ୍ରହ କରୁଛନ୍ତି। ସ୍ଥାନୀୟ ପ୍ରଶାସନ ମଧ୍ୟ ତାଙ୍କୁ ଏ ଦିଗରେ ସହଯୋଗ କରୁଛି। ଶିକ୍ଷା ଶେଷ କରି ପ୍ରଥମେ ସେ ନିଜ ଗାଁ ଗୋସାବାର ସରକାରୀ ସ୍କୁଲରେ ଅସ୍ଥାୟୀ ଶିକ୍ଷକ ଭାବେ ଯୋଗ ଦେଇଥିଲେ। ସେଥିତ୍ରେ ସ୍ଥାୟୀ ଶିକ୍ଷକ ନିଯୁକ୍ତ ହେବା ପରେ ୨୦୧୯ରେ ସେ ଚାକିରି ହରାଇଥିଲେ। ପରେ ସେ ପିଲାମାନଙ୍କୁ ମାଗଣାରେ ଟ୍ୟୁଶନ କରିବା ସହ ଏହି କାର୍ଯ୍ୟ ଆରମ୍ଭ କରିଥିଲେ, ଯାହାକୁ ଏଯାବତ୍ ସେ ଚାଲୁ ରଖିଛନ୍ତି।')

In [12]:
prev, now = round(len(link_lis) * 0.5), len(link_lis)
prev, now

(121620, 243240)

In [13]:
result = parallel(crawl, link_lis[prev:now], n_workers=16, progress=True)

In [14]:
create_op('output10.csv')

                                                 url  \
0  https://www.dharitri.com/hair-cut-for-trendy-l...   
1  https://www.dharitri.com/ଏବେ-ଆକ୍ସନ-ଫିଲ୍ମ-କରିପା...   
2  https://www.dharitri.com/rithvik-dhanjani-and-...   
3  https://www.dharitri.com/it-would-be-surprisin...   
4  https://www.dharitri.com/bollywood-actor-saif-...   

                                               title                time  \
0                   ଟ୍ରେଣ୍ଡି ଲୁକ୍‌ ପାଇଁ ହେୟାର୍‌ କଟ୍‌ 2021-05-25 12:58:00   
1                 ଏବେ ଆକ୍ସନ ଫିଲ୍ମ କରିପାରନ୍ତି ଶାହାରୁଖ 2019-09-07 15:14:00   
2  ୭ ବର୍ଷ ଚାଲିଥିଲା ପ୍ରେମ ସମ୍ପର୍କ; ହଠାତ୍‌ ଅଲଗା ହୋଇ... 2021-08-24 16:57:00   
3  ଶାହରୁଖଙ୍କ ଝିଅ ସୁହାନାଙ୍କ ଜୋତାର ମୂଲ୍ୟ ଜାଣିଲେ ହେବ... 2021-08-19 15:04:00   
4  ପ୍ରଭାସଙ୍କ ଫିଲ୍ମରେ ରାବଣ ଭୂମିକାରେ ନଜର ଆସିବେ ଏହି ... 2020-09-05 13:08:00   

                                                data  
0  ରାଉଣ୍ଡ ଫେସ୍: ଯେଉଁମାନଙ୍କ ମୁହଁ ଗୋଲ୍ ବା ରାଉଣ୍ଡ ସେ...  
1  ମୁମ୍ବାଇ: ବଲିଉଡ୍ ବାଦ୍ଶାହା ଶାହାରୁଖ ଖାଁ କିଛି ଦିନ ...  
2  ବମ୍ବେ: ଏକଦା ଅଭିନେତ୍ରୀ ଆଶା ନେଗି ଏବଂ ଅଭି

In [15]:
prev, now = 0, round(len(link_lis) * 0.2)
prev, now

(0, 48648)

In [16]:
result = parallel(crawl, link_lis[prev:now], n_workers=16, progress=True)

In [17]:
create_op('output2.csv')

                                                 url  \
0        https://www.dharitri.com/in-this-india-466/   
1    https://www.dharitri.com/survive-is-difficulty/   
2        https://www.dharitri.com/in-this-india-697/   
3  https://www.dharitri.com/explainer-of-dual-cul...   
4  https://www.dharitri.com/ପ୍ରତିବାଦ-ପ୍ରତିରୋଧ-ନୁହେଁ/   

                         title                time  \
0                    ଏଇ ଭାରତରେ 2022-01-09 07:03:00   
1               ବଞ୍ଚିବା କଷ୍ଟକର 2023-06-20 06:00:00   
2                    ଏଇ ଭାରତରେ 2022-09-19 07:07:00   
3  ଦ୍ୱୈତ ସଂସ୍କୃତିର ବ୍ୟାଖ୍ୟାକାର 2023-07-30 10:55:00   
4      ପ୍ରତିବାଦ ପ୍ରତିରୋଧ ନୁହେଁ 2023-12-12 06:13:00   

                                                data  
0  ପଶ୍ଚିମବଙ୍ଗର ସୁନ୍ଦରବନରେ ସଂକ୍ରମିତଙ୍କ ଚିକିତ୍ସା ଲା...  
1  ଗ୍ରୀଷ୍ମପ୍ରବାହ ଭାରତର ଅନେକ ରାଜ୍ୟରେ ଜନଜୀବନକୁ ଅସ୍ତ...  
2  କଦଳୀ ଉତ୍ପାଦନ ପରେ ଏହାର ଗଣ୍ଡିକୁ ଫିଙ୍ଗି ଦିଆଯାଉଥିବ...  
3  ଶଙ୍କରୀ ଚନ୍ଦ୍ରନ୍\n\nଏକ ଦଶନ୍ଧି ପୂର୍ବେ ଅଷ୍ଟ୍ରେଲିଆ...  
4  ଡ. ସତ୍ୟ ନାରାୟଣ ମିଶ୍ର\n\nସାମାଜିକ ଶୃଙ୍ଖଳିତ ଆଦର୍ଶ...  


In [19]:
ops = [pd.read_csv(fn) for fn in path.ls() if 'output' in fn.name]
op_df = pd.concat(ops, ignore_index=True)
op_df.head()

,url,title,time,data
0,https://www.dharitri.com/in-this-india-466/,ଏଇ ଭାରତରେ,2022-01-09 07:03:00,ପଶ୍ଚିମବଙ୍ଗର ସୁନ୍ଦରବନରେ ସଂକ୍ରମିତଙ୍କ ଚିକିତ୍ସା ଲା...
1,https://www.dharitri.com/survive-is-difficulty/,ବଞ୍ଚିବା କଷ୍ଟକର,2023-06-20 06:00:00,ଗ୍ରୀଷ୍ମପ୍ରବାହ ଭାରତର ଅନେକ ରାଜ୍ୟରେ ଜନଜୀବନକୁ ଅସ୍ତ...
2,https://www.dharitri.com/in-this-india-697/,ଏଇ ଭାରତରେ,2022-09-19 07:07:00,କଦଳୀ ଉତ୍ପାଦନ ପରେ ଏହାର ଗଣ୍ଡିକୁ ଫିଙ୍ଗି ଦିଆଯାଉଥିବ...
3,https://www.dharitri.com/explainer-of-dual-cul...,ଦ୍ୱୈତ ସଂସ୍କୃତିର ବ୍ୟାଖ୍ୟାକାର,2023-07-30 10:55:00,ଶଙ୍କରୀ ଚନ୍ଦ୍ରନ୍\n\nଏକ ଦଶନ୍ଧି ପୂର୍ବେ ଅଷ୍ଟ୍ରେଲିଆ...
4,https://www.dharitri.com/ପ୍ରତିବାଦ-ପ୍ରତିରୋଧ-ନୁହେଁ/,ପ୍ରତିବାଦ ପ୍ରତିରୋଧ ନୁହେଁ,2023-12-12 06:13:00,ଡ. ସତ୍ୟ ନାରାୟଣ ମିଶ୍ର\n\nସାମାଜିକ ଶୃଙ୍ଖଳିତ ଆଦର୍ଶ...


In [20]:
op_df.shape

(243240, 4)

In [21]:
op_df['article'] = op_df['title'].fillna('') + '\n' + op_df['data'].fillna('')
op_df['article'].head()

0    ଏଇ ଭାରତରେ\nପଶ୍ଚିମବଙ୍ଗର ସୁନ୍ଦରବନରେ ସଂକ୍ରମିତଙ୍କ ...
1    ବଞ୍ଚିବା କଷ୍ଟକର\nଗ୍ରୀଷ୍ମପ୍ରବାହ ଭାରତର ଅନେକ ରାଜ୍ୟ...
2    ଏଇ ଭାରତରେ\nକଦଳୀ ଉତ୍ପାଦନ ପରେ ଏହାର ଗଣ୍ଡିକୁ ଫିଙ୍ଗ...
3    ଦ୍ୱୈତ ସଂସ୍କୃତିର ବ୍ୟାଖ୍ୟାକାର\nଶଙ୍କରୀ ଚନ୍ଦ୍ରନ୍\n...
4    ପ୍ରତିବାଦ ପ୍ରତିରୋଧ ନୁହେଁ\nଡ. ସତ୍ୟ ନାରାୟଣ ମିଶ୍ର\...
Name: article, dtype: object

In [34]:
op_df['article'] = op_df['article'].str.replace(pttr['mobile_ptrn'], ' ', regex=True)

In [41]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [43]:
print(op_df[op_df['url'] == 'https://www.dharitri.com/କରୋନା-ଆଶୀର୍ବାଦର-ରୂପ-ନେଉ/']['article'])

859    କରୋନା: ଆଶୀର୍ବାଦର ରୂପ ନେଉ\nକରୋନା ଭୂତାଣୁର ଦ୍ରୁତ ବିସ୍ତାର ବିଶ୍ୱକୁ ଅସ୍ତବ୍ୟସ୍ତ କରିଦେଇଛି I ଚାଇନାର ଉହାନ ସହରରୁ ଆରମ୍ଭ ହୋଇ ଏହା ପାଞ୍ଚମାସ ଭିତରେ ବିଶ୍ୱକୁ ନିଜ ପଞ୍ଝାରେ ବାନ୍ଧି ରଖିପାରିଛି I ଏହାଦ୍ୱାରା ବିଶ୍ୱର ବଡବଡ ସମୃଦ୍ଧଶାଳୀ ରାଷ୍ଟ୍ରମାନେ ଏକପ୍ରକାର ଉପାୟଶୂନ୍ୟ I  \nଭାରତରେ ଏହାର ପ୍ରକୋପ ବଢିନାହିଁ ଏହା ନିଶ୍ଚୟ ଭଗବାନଙ୍କର ଆଶୀର୍ବାଦ I କାରଣ ଭାରତ ଭଳି ଏକ ଜନବହୁଳ ଦେଶରେ ଏହା ଦ୍ରୁତଗତିରେ ବ୍ୟାପିବାର ଆଶଙ୍କା ନିଶ୍ଚୟ ରହିଛି I ବିଶେଷଭାବରେ ଭାରତରେ ସ୍ବାସ୍ଥ୍ୟ ବ୍ୟବସ୍ଥାର ଯେଉଁ ସ୍ଥିତି ତାହା କରୋନା ସଂକ୍ରମଣକୁ ରୋକିବାରେ କେତେଦୂର ସମର୍ଥ ତାହା ମଧ୍ୟ ଆମେ ଊଣା ଅଧିକେ ଜାଣି ସାରିଲେଣି I  \nତନ୍ମଧ୍ୟରେ ଆଉ ଗୋଟିଏ କଥା ଆଖି ଆଗରେ ଜଳଜଳ ହୋଇ ଦିଶୁଛି; ତାହା ହେଲା ଆମ ଦେଶର ଲୋକଙ୍କର ସଚେତନତା I ବିନା କାରଣରେ ବିପଦକୁ ଡାକିବା ପାଇଁ ଆମ ଦେଶର ଅନେକ ଲୋକ ‘ତାଲା ବନ୍ଦ’ ନିୟମକୁ ଉଲ୍ଲଂଘନ କରିବା ପରିଲକ୍ଷିତ ହେଉଛି I  \nଯଦିଓ ଏ କ୍ଷେତ୍ରରେ ଅତ୍ୟାବଶ୍ୟକ ଦ୍ରବ୍ୟ ସଂଗ୍ରହ ଏକ ଗୁରୁତ୍ୱପୂର୍ଣ୍ଣ ବିଷୟ କିନ୍ତୁ ବାହାରେ ବୁଲୁଥିବା ସମସ୍ତ ବ୍ୟକ୍ତିବିଶେଷଙ୍କର ଯେ ସେଇ ଗୋଟିଏ ଲକ୍ଷ୍ୟ ତାହା ଅସମ୍ଭବ I ବିନା କାମରେ ବାହାରେ ବୁଲୁଥିବା ବ୍ୟକ୍ତିମାନେ ନିଶ୍ଚୟ ଦେଶଦ୍ରୋହୀ I କିନ୍ତୁ ବାହାରେ ବୁଲୁଥିବା ସେହିଭଳି ବ୍ୟକ୍ତିମାନଙ୍କ ଭିତରେ ରହିଯାଇଛନ୍ତି କିଛି ଏଭଳି ବ୍ୟକ୍ତି ଯେଉଁମାନଙ୍

In [22]:
op_df = op_df[op_df['article'] != '\n']
op_df.shape

(243231, 5)

In [44]:
op_df.to_csv("article.csv", index=False)

In [26]:
!du -h ar*

1.6G	article.csv


In [ ]:
24000/12

375.0